## 1. Service
- Service 有两种启动方式，并且它的两种启动方式的生命周期是不一样的

![image](page1.png)

![image](page2.png)

---

### 1.1 startService 方式启动 Service
- 一旦服务启动，它就会在后台无限期的运行，生命周期独立于启动它的组件，即使启动它的组件已经销毁了也不受任何影响，由于启动的服务长期运行在后台，这会大量消耗手机的电量，因此，我们应该在任务执行完成之后调用 stopSelf() 来停止服务，或者通过其他应用组件调用 stopService 来停止服务。
- 生命周期：onCreate() －> onStartCommand() －> onStart()(现在已经废弃) －> onDestroy()
  - onCreate(): 首次启动服务的时候，系统会调用这个方法，在 onStartCommand 和 onBind 方法之前，如果服务已经启动起来了，再次启动时，则不会调用此方法。可以在 onCreate 方法中做一些初始化的操作，比如要执行耗时的操作，可以在这里创建线程，要播放音乐，可以在这里初始化音乐播放器。
  - onStartCommand(): 当通过 startService 方法来启动服务的时候，在 onCreate 方法之后就会回调这个方法，此方法调用后，服务就启动起来了，将会在后台无限期的运行，直到通过 stopService 或者 stopSelf 方法来停止服务。
  - onDestroy(): 当服务不再使用且将被销毁时，系统将调用此方法。服务应该实现此方法来清理所有资源，如线程、注册的侦听器、接收器等。这是服务接收的最后一个调用。


- *小结：通过 startService 方式启动的服务，服务会无限期的在后台运行，直到通过 stopService 或 stopSelf 来终止服务。服务独立于启动它的组件， 当组件启动服务后，组件和服务就再也没有关系了，就算启动它的组件被销毁了，服务照样在后台运行。通过这种方式启动的服务不好与组件之间通信。*

---

### 1.2 bindService 方式启动 Service
- 绑定即将启动组件和服务绑定在一起。多个组件可以绑定到同一个服务上，如果只有一个组件绑定服务，当绑定的组件被销毁时，服务也就会停止了。如果是多个组件绑定到一个服务上，当绑定到该服务的所有组件都被销毁时，服务才会停止。
- 生命周期如下：onCreate －> onBind －> onUnbind －> onDestroy。其中重要的就是 onBind 和 onUnbind 方法
  - onBind(): 当其他组件想通过 bindService 与服务绑定时，系统将会回调这个方法，在实现中，你必须返回一个 IBinder 接口，供客户端与服务进行通信，必须实现此方法，这个方法是 Service 的一个抽象方法，但是如果你不允许绑定的话，返回 null 就可以了
  - onUnbind(): 当所有与服务绑定的组件都解除绑定时，就会调用此方法
  
#### demo
- 添加一个类继承 Binder, 在 Binder 类中添加其他组件要与服务交互的方法，并在 onBind() 方法中返回 IBinder 实例对象：
```
public class SimpleService extends Service {
    public static final String TAG = "SimpleService";
    
    @Nullable
    @Override
    public IBinder onBind(Intent intent) {
        Log.i(TAG,"call onBind...");
        // 返回 IBinder 接口对象
        return new MyBinder();
    }
 
    @Override
    public boolean onUnbind(Intent intent) {
        Log.i(TAG,"call onUnbind...");
        return super.onUnbind(intent);
    }
 
    @Override
    public void onCreate() {
        Log.i(TAG,"call onCreate...");
    }
 
    @Override
    public void onStart(Intent intent, int startId) {
        Log.i(TAG,"call onStart...");
    }
 
    @Override
    public int onStartCommand(Intent intent, int flags, int startId) {
        Log.i(TAG,"call onStartCommand...");
        return super.onStartCommand(intent, flags, startId);
    }
 
    @Override
    public void onDestroy() {
        Log.i(TAG,"call onDestroy...");
    }
 
    // 添加一个类继承 Binder
    public class MyBinder extends Binder {
        // 添加要与外界交互的方法
        public String getStringInfo() {
            return "调用了服务中的方法";
        }
    }
}
```

- 绑定服务的时候，需要提供一个 ServiceConnection 接口，在接口回调中获取 Binder 对象，与服务进行通信

```
private SimpleService.MyBinder mMyBinder;

// 绑定/解除绑定 Service 回调接口
private ServiceConnection mConnection = new ServiceConnection() {
    @Override
    public void onServiceConnected(ComponentName name, IBinder service) {
        // 绑定成功后回调
        // 1, 获取Binder接口对象
        mMyBinder = (SimpleService.MyBinder) service;
        // 2, 从服务获取数据
        String content = mMyBinder.getStringInfo();
        // 3,界面提示
        Toast.makeText(ServiceSimpleActivity.this,content,Toast.LENGTH_LONG).show();
    }

    @Override
    public void onServiceDisconnected(ComponentName name) {
       // 解除绑定后回调
        mMyBinder = null;
    }
};
```

- 绑定和解除绑定服务

```
case R.id.bind_service:
    Intent intent = new Intent(this, SimpleService.class);
    // 绑定服务
    bindService(intent, mConnection, Context.BIND_AUTO_CREATE);
    break;
case R.id.unbind_service:
    // 解除绑定服务
    unbindService(mConnection);
    break;
```

- tips: Service 的生命周期方法不同于 Activity ，不需要调用超类的生命周期方法，如：不用调用 super.onCreate()

---

### 1.3 多个组件绑定同一服务
- Service 是支持多个组件绑定在同一个服务上的，第一个组件绑定是会回调 onCreate 生命周期方法，后续的绑定只会调用 onBind 方法，返回 IBinder 给客户端。
- 当绑定在服务上的组件都调用 unbindService 解除绑定服务或者组件本身就已经被系统回收，那么服务也就会被停止回收了，会回调 onUnbind 和 onDestroy 方法

---

## 2. Service 与应用组件通信的几种方式
### 2.1 BroadcastReceiver
- 场景描述：通过 startService 启动一个长期在后台运行的下载图片服务，然后在界面上点击下载按钮，通过 intent 传递一个下载链接给 Service，在下载完成后，通过 BroadcastReceiver 通知 Activity 界面显示图片。
```
public class DownloadService extends Service {
    public static final String IMAGE = "image_url";
    public static final String RECEIVER_ACTION = "com.demo.simpleservice";
    private static final String TAG = "DownloadService";
    public static final String ACTION_START_SERVICER = "com.demo.startservice";
    public static final String ACTION_DOWNLOAD = "com.demo.startdownload";
    private Looper mServiceLooper;
    private ServiceHandler mServiceHandler;
    private final class ServiceHandler extends Handler {
        public ServiceHandler(Looper looper){
            super(looper);
        }
 
        @Override
        public void handleMessage(Message msg) {
            // 工作线程做耗时下载
            String url = (String) msg.obj;
            Bitmap bitmap = null;
            try {
                bitmap = Picasso.with(getApplicationContext()).load(url).get();
                Intent intent = new Intent();
                intent.putExtra("bitmap", bitmap);
                intent.setAction(RECEIVER_ACTION);
                // 通知显示
                LocalBroadcastManager.getInstance(getApplicationContext()).sendBroadcast(intent);
            } catch (IOException e) {
                e.printStackTrace();
            }
 
            // 工作完成之后，停止服务
            stopSelf();
        }
    }
    
    @Nullable
    @Override
    public IBinder onBind(Intent intent) {
        return null;
    }
    
    @Override
    public void onCreate() {
        // 开启一个工作线程做耗时工作
        HandlerThread thread = new HandlerThread("ServiceHandlerThread", Process.THREAD_PRIORITY_BACKGROUND);
        thread.start();
        
        // 获取工作线程的 Looper
        mServiceLooper = thread.getLooper();
        
        // 创建工作线程的 Handler
        mServiceHandler = new ServiceHandler(mServiceLooper);
    }
 
    @Override
    public int onStartCommand(Intent intent, int flags, int startId) {
        Log.i(TAG,"call onStartCommand...");
        if (intent.getAction().equals(ACTION_DOWNLOAD)) {
            handleCommand(intent);
        } else if (intent.getAction().equals(ACTION_START_SERVICER)) {
            // do nothing
        }
 
        return START_STICKY;
    }
 
    private void handleCommand(Intent intent) {
        String url = intent.getStringExtra(IMAGE);
        // 发送消息下载
        Message message = mServiceHandler.obtainMessage();
        message.obj = url;
        mServiceHandler.sendMessage(message);
    }
}
```

---

### 2.2 使用 Binder 和 服务通信
- 模拟一个用服务播放音乐的例子来讲一下组件通过 Binder 接口和服务之间通信

- 定义一个通信的接口 IPlayer

```
public interface IPlayer {
    // 播放
    public void play();
    // 暂停
    public void pause();
    // 停止
    public void stop();
    // 获取播放进度
    public int getProgress();
    // 获取时长
    public int getDuration();
}
```

- 添加一个 MusicService 类，继承 Service 实现 Iplayer 接口
  - 比较重要的就是内部类 LocalService ，继承Binder ,里面提供一个getService 方法，返回MusicService 实例，组件通过IBinder 获取到Music 实例后，就可以和Service之间相互通信

```
public class MusicService extends Service implements IPlayer {
    public static final String TAG = "MusicService";
    
    private LocalService mBinder = new LocalService();
    
    public class LocalService extends Binder {
        public MusicService getService() {
            return MusicService.this;
        }
    }

    @Nullable
    @Override
    public IBinder onBind(Intent intent) {
        return mBinder;
    }
 
    @Override
    public void play() {
        Log.i(TAG,"music play...");
    }
 
    @Override
    public void pause() {
        Log.i(TAG,"music pause...");
    }
 
    @Override
    public void stop() {
        Log.i(TAG,"music stop...");
    }
 
    @Override
    public int getProgress() {
        return 100;
    }
 
    @Override
    public int getDuration() {
        return 10240;
    }
}
```

- Activity 中代码如下

```
private MusicService.LocalService mLocalService;
private MusicService mMusicService;

// 绑定/解除绑定 Service 回调接口
private ServiceConnection mConnection = new ServiceConnection() {
    @Override
    public void onServiceConnected(ComponentName name, IBinder service) {
        // 1, 获取 Binder 接口对象
        mLocalService = (MusicService.LocalService) service;
        
        // 2, 获取 MusicService 实例
        mMusicService = mLocalService.getService();

        // 只要拿到 Music Service 实例之后，就可以调用接口方法了
        // 可以通过它来播放／暂停音乐，还可以通过它来获取当前播放音乐的进度，时长等等

        mMusicService.play();
        mMusicService.pause();
        mMusicService.stop();

        int progress = mMusicService.getProgress();
        Log.i(MusicService.TAG, "progress:" + progress);

        int duration = mMusicService.getDuration();
        Log.i(MusicService.TAG, "duration:" + duration);
    }

    @Override
    public void onServiceDisconnected(ComponentName name) {
       // 解除绑定后回调
        mMusicService = null;
        mLocalService = null;
    }
};
```

---

### 2.3 使用 AIDL 通信(跨进城通信IPC)
- AIDL（Android 接口定义语言）执行所有将对象分解成原语的工作，操作系统可以识别这些原语并将它们编组到各进程中，以执行 IPC。

---

## 3. Service 总结
- Service 有2种启动方式
- Service 是运行在主线程中的，因此不能执行耗时的或者密集型的任务，如果要执行耗时操作或者密集型计算任务，请在服务中开启工作线程，在线程中执行。或者使用下面介绍的的 IntentService。

---

## 4. IntentService
- IntentService 是 Service 的子类，它使用工作线程逐一处理所有启动请求，如果不要求服务同时处理多个请求，这是最好的选择。只需实现 onHandIntent 方法即可，该方法会接收每个启动请求的 Intent，使您能够执行后台工作。

### 4.1 IntentService 示例
- IntentService 默认为我们开启了一个工作线程，在任务执行完毕后，自动停止服务，因此在我们大多数的工作中，使用 IntentService 就够了，并且IntentService 比较简单，只要实现一个方法 OnHandleIntent

```
public class MyIntentService extends IntentService {
    public static final String TAG = "MyIntentService";
    /**
     * Creates an IntentService. Invoked by your subclass's constructor.
     *
     * @param name Used to name the worker thread, important only for debugging.
     */
    public MyIntentService() {
        super("MyIntentService");
    }
 
    @Override
    protected void onHandleIntent(@Nullable Intent intent) {
        // 这里已经是工作线程，在这里执行操作就行
        boolean isMainThread = Thread.currentThread() == Looper.getMainLooper().getThread();
        Log.i(TAG, "is main thread:" + isMainThread);
 
        // 执行耗时下载操作
        mockDownload();
    }

    /**
     * 模拟执行下载
     */
    private void mockDownload() {
       try {
           Thread.sleep(5000);
           Log.i(TAG,"下载完成...");
       } catch (Exception e) {
           e.printStackTrace();
       }
    }
}
```

---

### 4.2 IntentService 源码浅析

```
// 1, 有一个 Looper 变量和一个 ServiceHandler 变量，ServiceHander 继承 Handler 处理消息
private volatile Looper mServiceLooper;
private volatile ServiceHandler mServiceHandler;
private String mName;
private boolean mRedelivery;

private final class ServiceHandler extends Handler {
    public ServiceHandler(Looper looper) {
        super(looper);
    }

    @Override
    public void handleMessage(Message msg) {
        // 在工作线程中调用 onHandleIntent，子类根据 Intent 传递的数据执行具体的操作
        onHandleIntent((Intent)msg.obj);
        
        // 任务执行完毕后，自动停止 Service
        stopSelf(msg.arg1);
    }
}

// 2, 在 onCreate 方法中，创建了一个线程 HandlerThread，并启动线程
// 然后获取工作线程的 Looper ，并用 Looper 初始化 Handler(我们都知道 Handler 的创建需要一依赖 Looper)
public void onCreate() {
    // TODO: It would be nice to have an option to hold a partial wakelock
    // during processing, and to have a static startService(Context, Intent)
    // method that would launch the service & hand off a wakelock.

    super.onCreate();
    HandlerThread thread = new HandlerThread("IntentService[" + mName + "]");
    thread.start();

    mServiceLooper = thread.getLooper();
    mServiceHandler = new ServiceHandler(mServiceLooper);
}

// 3, 在 onStart() 方法中发送消息给 Handler,并且把 Intent 传给了 Handler 处理
@Override
public void onStart(@Nullable Intent intent, int startId) {
    Message msg = mServiceHandler.obtainMessage();
    msg.arg1 = startId;
    msg.obj = intent;
    mServiceHandler.sendMessage(msg);
}

// 4，onStartCommand 直接调用的是 onStart 方法
public int onStartCommand(@Nullable Intent intent, int flags, int startId) {
    onStart(intent, startId);
    return mRedelivery ? START_REDELIVER_INTENT : START_NOT_STICKY;
}

// 5 最后就是一个子类需要实现的抽象方法，这个方法在 handleMessage 中调用，也就是在工作线程中执行。
protected abstract void onHandleIntent(@Nullable Intent intent);
```

---

### 4.3 IntentService 总结
- 如果要绑定服务的话，还是要重写 onBind 返回一个 IBinder 的
- 使用 Service 可以同时执行多个请求，而使用 IntentService 只能同时执行一个请求。

![image](page3.webp)

---